<a href="https://colab.research.google.com/github/rushiai/Advance-House-Price/blob/main/24_5_YT_all_text_%2B_summary_%2B_audio_%2B_wrong_images_%2B_video_%2B_we_need_headline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS
os.chdir("TensorFlowTTS")
!pip install  .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")
print("Downloading Tacotron2 model...")
!gdown --id {"12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW"} -O tacotron2-120k.h5
!gdown --id {"1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY"} -O tacotron2_config.yml
print("Downloading MelGAN-STFT model...")
!gdown --id {"1WB5iQbk9qB-Y-wO8BU6S2TnRiu4VU5ys"} -O melgan.stft-2M.h5
!gdown --id {"1OqdrcHJvtXwNasEZP7KXZwtGUDXMKNkg"} -O melgan.stft_config.yml
print("Downloading MelGAN model...")
!gdown --id {"1A3zJwzlXEpu_jHeatlMdyPGjn1V7-9iG"} -O melgan-1M6.h5
!gdown --id {"1Ys-twSd3m2uqhJOEiobNox6RNQf4txZs"} -O melgan_config.yml
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import AutoProcessor

tacotron2_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/tacotron2/conf/tacotron2.v1.yaml')
tacotron2 = TFAutoModel.from_pretrained(
    config=tacotron2_config,
    pretrained_path="tacotron2-120k.h5",
    name="tacotron2"
)

melgan_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/melgan/conf/melgan.v1.yaml')
melgan = TFAutoModel.from_pretrained(
    config=melgan_config,
    pretrained_path="melgan-1M6.h5",
    name="melgan"
)

melgan_stft_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/melgan_stft/conf/melgan_stft.v1.yaml')
melgan_stft = TFAutoModel.from_pretrained(
    config=melgan_stft_config,
    pretrained_path="melgan.stft-2M.h5",
    name="melgan_stft"
)

print("Downloading ljspeech_mapper.json ...")
!gdown --id {"1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e"} -O ljspeech_mapper.json

processor = AutoProcessor.from_pretrained(pretrained_path="./ljspeech_mapper.json")

def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH":
    mel_before, mel_outputs, duration_outputs = text2mel_model.inference(
        input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MELGAN" or vocoder_name == "MELGAN-STFT":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  elif vocoder_name == "MB-MELGAN":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MELGAN, MELGAN-STFT and MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()

def visualize_attention(alignment_history):
  import matplotlib.pyplot as plt

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)
  ax.set_title(f'Alignment steps')
  im = ax.imshow(
      alignment_history,
      aspect='auto',
      origin='lower',
      interpolation='none')
  fig.colorbar(im, ax=ax)
  xlabel = 'Decoder timestep'
  plt.xlabel(xlabel)
  plt.ylabel('Encoder timestep')
  plt.tight_layout()
#   plt.show()
  plt.close()

def visualize_mel_spectrogram(mels):
  mels = tf.reshape(mels, [-1, 80]).numpy()
  fig = plt.figure(figsize=(10, 8))
  ax1 = fig.add_subplot(311)
  ax1.set_title(f'Predicted Mel-after-Spectrogram')
  im = ax1.imshow(np.rot90(mels), aspect='auto', interpolation='none')
  fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
#   plt.show()
  plt.close()

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10380, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 10380 (delta 43), reused 42 (delta 17), pack-reused 10290
Receiving objects: 100% (10380/10380), 131.44 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (4983/4983), done.
Processing /content/TensorFlowTTS
     |████████████████████████████████| 320.4MB 47kB/s 
     |████████████████████████████████| 686kB 30.6MB/s 
     |████████████████████████████████| 245kB 40.9MB/s 
     |████████████████████████████████| 225kB 39.1MB/s 
     |████████████████████████████████| 3.6MB 29.2MB/s 
     |████████████████████████████████| 1.3MB 29.3MB/s 
     |████████████████████████████████| 1.7MB 19.5MB/s 
     |████████████████████████████████| 3.1MB 26.5MB/s 
  Cloning https://github.com/repodiac/german_transliterate.git to /tmp/pip-install-kypb6roq/german-transliterate
  Running command git clone -q https://github.com/r

Downloading...
From: https://drive.google.com/uc?id=12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW
To: /content/tacotron2-120k.h5
128MB [00:01, 89.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY
To: /content/tacotron2_config.yml
100% 1.49k/1.49k [00:00<00:00, 4.32MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WB5iQbk9qB-Y-wO8BU6S2TnRiu4VU5ys
To: /content/melgan.stft-2M.h5
17.1MB [00:00, 47.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OqdrcHJvtXwNasEZP7KXZwtGUDXMKNkg
To: /content/melgan.stft_config.yml
100% 1.77k/1.77k [00:00<00:00, 2.72MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A3zJwzlXEpu_jHeatlMdyPGjn1V7-9iG
To: /content/melgan-1M6.h5
17.1MB [00:00, 41.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ys-twSd3m2uqhJOEiobNox6RNQf4txZs
To: /content/melgan_config.yml
100% 1.29k/1.29k [00:00<00:00, 1.10MB/s]
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...


In [2]:
!apt update &> /dev/null
!apt install imagemagick &> /dev/null
!apt install ffmpeg &> /dev/null
!pip3 install moviepy[optional] &> /dev/null
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

In [3]:
!pip install transformers
!pip install newspaper3k
!pip install kora -q
from kora.selenium import wd
import nltk
nltk.download('punkt')

     |████████████████████████████████| 2.3MB 10.9MB/s 
     |████████████████████████████████| 901kB 36.4MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 
     |████████████████████████████████| 215kB 9.5MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 7.4MB 10.0MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=bdcccb21f4447becaede37370e3f16e25e6010b48fbec8db392b9c907cbee7d3
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398406 sha256=33d1ddddbcb35a9d82501e8484f3904b80a2115b8dcf21ec474e6863be9b35cd
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl siz

True

In [4]:
pip install path

In [5]:
import time
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import glob
import shutil
import numpy as np
import re
from newspaper import Article

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from transformers import pipeline

In [6]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from kora.selenium import wd
import os
import scipy
from scipy.io.wavfile import write
import glob
import shutil
import textwrap
# from resizeimage import resizeimage
from moviepy.editor import * # ImageClip
from PIL import Image
import os
from path import Path
import numpy as np
import cv2

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1523712/45929032 bytes (3.3%)4505600/45929032 bytes (9.8%)7520256/45929032 bytes (16.4%)10518528/45929032 bytes (22.9%)13819904/45929032 bytes (30.1%)17137664/45929032 bytes (37.3%)20373504/45929032 bytes (44.4%)23707648/45929032 bytes (51.6%)27107328/45929032 bytes (59.0%)30072832/45929032 bytes (65.5%)33226752/45929032 bytes (72.3%)36306944/45929032 bytes (79.1%)3

In [7]:
summarizer = pipeline("summarization", device=0)
# summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)

In [8]:
categories = ['b','e']
countries = ['US','CA']
master_dic = {}

In [9]:
for count in countries:
    master_dic[count] = {}
    for cat in categories:
        df = pd.DataFrame()
        dfa = pd.DataFrame()
        link = 'https://trends.google.com/trends/trendingsearches/realtime?geo={0}&category={1}'.format(count,cat)
        master_dic[count][cat] = {}
        wd.get(link)
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        page = wd.page_source
        soup = BeautifulSoup(page, 'html.parser')
        result = soup.find("article")
        head_results = soup.find_all("div", class_="summary-text")
        regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        for i in head_results:
            # print(i)
            for j in i:
                # print(j)
                linkss = re.findall(regex,str(j))
                if len(linkss) != 0:
                    # print(re.findall(regex,str(linkss)))
                    z = re.findall(regex,str(linkss))
                    k = pd.Series(z)
                    df = df.append(k,ignore_index=True)

        for i in range(0,len(df)):
            a = pd.Series(df[0][i][:-2])
            dfa = dfa.append(a,ignore_index=True)
        dfa['text'] = np.nan
        for i in range(0,len(df)):
            url = dfa[0][i]
            article = Article(url)
            try:
                article.download()
                article.parse()
                article.nlp()
                art_text = article.text
                s = pd.Series(art_text)
                # dfa['text'][i] = dfa.append(s,ignore_index=True)
                dfa['text'][i] = art_text
            except:
                print('error ',i)
        dfa = dfa.dropna()
        master_dic[count][cat] = dfa

error  1
error  8
error  9
error  10
error  1
error  5
error  11
error  13


In [10]:
master_dic

{'CA': {'b':                                                     0                                               text
  0   https://analyticsindiamag.com/top-countries-pu...  The quantum computing market is valued at $472...
  2   https://www.cbc.ca/news/canada/london/open-clo...  With a provincial lockdown still in effect, mo...
  3   https://neighborwebsj.com/uncategorized/671473...  White Vinegar Market Research Study The explor...
  4   https://www.thejournal.ie/virgin-galactic-shut...  A VIRGIN GALACTIC shuttle with a crew on board...
  6   https://www.farminguk.com/news/wind-turbines-b...  Farmers and landowners who have previously bee...
  7   https://www.whatech.com/markets-research/it/70...  Context Aware Computing Market 2021 size, grow...
  8   https://www.globenewswire.com/news-release/202...  Pune, India, May 24, 2021 (GLOBE NEWSWIRE) --\...
  9   https://www.teslarati.com/tesla-cybertruck-for...  By\n\nThe Ford F-150 Lightning debuted on May ...
  10  https://swarajyamag.

In [11]:
for count in countries:
    for cat in categories:
        master_dic[count][cat]['summary'] = np.nan
        master_dic[count][cat]['headline'] = np.nan
        for i in range(0,len(master_dic[count][cat]['text'])):
            try:
                if len(master_dic[count][cat]['text'][i]) > 300:
                    try:
                        aaa = summarizer(master_dic[count][cat]['text'][i][:2000], max_length=400, min_length=100, do_sample=False)
                        bbb = summarizer(master_dic[count][cat]['text'][i][:2000], max_length=25, min_length=10, do_sample=False)
                        # bbb = summarizer(aaa[0]['summary_text'], max_length=25, min_length=10, do_sample=False)
                        master_dic[count][cat]['summary'][i] = aaa[0]['summary_text']
                        master_dic[count][cat]['headline'][i] = bbb[0]['summary_text']
                    except:
                        print('*********** {0}****{1}'.format(count,cat))
            except: print('*****************************************************')
        master_dic[count][cat].dropna(inplace=True)
        master_dic[count][cat].reset_index(inplace=True,drop=True)

*****************************************************


Your max_length is set to 400, but you input_length is only 249. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************


Your max_length is set to 400, but you input_length is only 372. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 345. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 390. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 329. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 347. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************
*****************************************************


Your max_length is set to 400, but you input_length is only 359. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************


Your max_length is set to 400, but you input_length is only 340. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************


Your max_length is set to 400, but you input_length is only 387. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 347. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 329. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [12]:
master_dic

{'CA': {'b':                                                    0  ...                                           headline
  0  https://analyticsindiamag.com/top-countries-pu...  ...   The quantum computing market is valued at $47...
  1  https://www.cbc.ca/news/canada/london/open-clo...  ...   Most London, Ont., retail stores are closed o...
  2  https://neighborwebsj.com/uncategorized/671473...  ...   Government and public solicitation of ideas t...
  3  https://www.thejournal.ie/virgin-galactic-shut...  ...   The VSS Unity's rocket motor sent out the shi...
  4  https://www.farminguk.com/news/wind-turbines-b...  ...   Policy change could mean onshore wind will so...
  5  https://www.whatech.com/markets-research/it/70...  ...   Context Aware Computing Market 2021 size, gro...
  6  https://www.globenewswire.com/news-release/202...  ...   Mobile banking have gained a lot of importanc...
  7  https://www.teslarati.com/tesla-cybertruck-for...  ...   The Ford F-150 Lightning debuted on May

In [13]:
master_dic['CA']['b']

,0,text,summary,headline
0,https://analyticsindiamag.com/top-countries-pu...,The quantum computing market is valued at $472...,The quantum computing market is valued at $47...,The quantum computing market is valued at $47...
1,https://www.cbc.ca/news/canada/london/open-clo...,"With a provincial lockdown still in effect, mo...","Most London, Ont., retail stores are closed o...","Most London, Ont., retail stores are closed o..."
2,https://neighborwebsj.com/uncategorized/671473...,White Vinegar Market Research Study The explor...,Government and public solicitation of ideas t...,Government and public solicitation of ideas t...
3,https://www.thejournal.ie/virgin-galactic-shut...,A VIRGIN GALACTIC shuttle with a crew on board...,The VSS Unity's rocket motor sent out the shi...,The VSS Unity's rocket motor sent out the shi...
4,https://www.farminguk.com/news/wind-turbines-b...,Farmers and landowners who have previously bee...,Policy change could mean onshore wind will so...,Policy change could mean onshore wind will so...
5,https://www.whatech.com/markets-research/it/70...,"Context Aware Computing Market 2021 size, grow...","Context Aware Computing Market 2021 size, gro...","Context Aware Computing Market 2021 size, gro..."
6,https://www.globenewswire.com/news-release/202...,"Pune, India, May 24, 2021 (GLOBE NEWSWIRE) --\...",Mobile banking have gained a lot of importanc...,Mobile banking have gained a lot of importanc...
7,https://www.teslarati.com/tesla-cybertruck-for...,By\n\nThe Ford F-150 Lightning debuted on May ...,"The Ford F-150 Lightning debuted on May 19, a...","The Ford F-150 Lightning debuted on May 19, a..."


In [15]:
# !pip install kora -q
# !pip install bing-image-downloader
from kora.selenium import wd
from bs4 import BeautifulSoup
import time
import re
import requests
def image_download(query,path='/dataset/',max=5,show=True):
    try:
        wd.get("https://www.google.com/search?q={0}&rlz=1C1CHBF_enIN922IN922&sxsrf=ALeKk0378KGZuab4G3o516deoY5ZqW-xbg:1620111097016&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiGmLyTuK_wAhVuwzgGHfChD8kQ_AUoAXoECAIQAw&biw=1366&bih=635".format(query))
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if max >= 75:
            time.sleep(5)
            wd.execute_script("window.scrollTo(15, document.body.scrollHeight);")
            time.sleep(5)
        page = wd.page_source
        soup = BeautifulSoup(page, 'html.parser')
        # soup.find_all(re.compile("(?:http\:|https\:)?\/\/.*\.(?:jpg)"))
        obj = re.findall('(?:http\:|https\:)?\/\/.*\.(?:jpg)',str(soup))
        print(len(obj))
        directory = os.getcwd()+'{0}{1}'.format(path,query)
        if not os.path.isdir(directory):
            os.mkdir(directory)
        for i in range(3,max+3):
            if show == True:
                print('downloading img no ',i-3)
            response = requests.get(obj[i])
            file = open("{0}/{1}{2}.jpg".format(directory,query,i), "wb")
            file.write(response.content)
            file.close()
        print('download {} images'.format(max))
    except OSError as error: 
        print(error)

In [18]:
try:
    os.mkdir('output')
except:
    print('already there')
fol = ['img','video','final_video','dataset']
for count in countries:
    for cat in categories:
        dfx = master_dic[count][cat]
        for y in range(0,len(fol)):
            a = fol[y]
            try:
                shutil.rmtree(a)
            except:
                print('error folder delete {}'.format(a))
            try:
                os.mkdir(a)
            except:
                print('error folder {}'.format(a))
        dfx = dfx.head(5)
        for i in range(0,len(dfx)):
            input_text = dfx.headline[i] + '!!!,' + dfx.summary[i]
            mels, alignment_history, audios = do_synthesis(input_text, tacotron2, melgan_stft, "TACOTRON", "MELGAN-STFT")
            scipy.io.wavfile.write('audios.mp3', rate=22050, data=audios)
            audio_clip = AudioFileClip('/content/audios.mp3')
            # time.sleep(5)
            try:
                image_download(dfx.headline[i],max=10)
            except:
                print('image download error ',dfx.headline[i])
            # downloader.download(dfx.headline[i], limit=10,  output_dir='dataset', adult_filter_off=True, force_replace=False, timeout=60)
            try:
                this_dir = os.listdir('/content/dataset/{}'.format(dfx.headline[i]))
                filepaths = [os.path.join('/content/dataset/{}'.format(dfx.headline[i]), fname) for fname in this_dir if fname.endswith("jpg")]
            except:
                print('image folder error')
            dir = '/content/img'
            for f in os.listdir(dir):
                os.remove(os.path.join(dir, f))
            for j in filepaths:
                try:
                    image = cv2.imread(j, 1)
                    stretch_near = cv2.resize(image, (1280, 720),interpolation = cv2.INTER_NEAREST)
                    cv2.imwrite('/content/img/{}'.format(j[-11:]), stretch_near)
                except:
                    print('resize error')
            if len(glob.glob('/content/img/*.jpg')) > 0:
                this_dir = os.listdir('/content/img')
                filepaths = [os.path.join('/content/img', fname) for fname in this_dir if fname.endswith("jpg")][:5]
                output_video = os.path.join('/content/video', 'video{}.mp4'.format(i))
                # clip = ImageSequenceClip(filepaths, fps=0.2)
                clip = ImageSequenceClip(filepaths, fps=0.25)
                clip.write_videofile(output_video, codec='libx264', audio_codec="aac")

                final_video_path = os.path.join('/content/final_video', 'final-video{}.mp4'.format(i))
                video_clip = VideoFileClip(output_video)

                final_clip = video_clip.set_audio(audio_clip)
                final_clip.write_videofile(final_video_path, codec='libx264', audio_codec="aac")

                tclip = VideoFileClip(final_video_path)

                out = ''
                wrapper = textwrap.TextWrapper(width=18)
                word_list = wrapper.wrap(text=dfx.headline[i])
                for element in word_list:
                    out = out + element + '\n'

                # Generate a text clip
                txt_clip = TextClip(out, fontsize = 65, color = 'black', font="Amiri-Bold")
                    
                # setting position of text in the center and duration will be 10 seconds
                txt_clip = txt_clip.set_pos('center',relative=True).set_duration(5)
                    
                # Overlay the text clip on the first video clip
                video = CompositeVideoClip([tclip, txt_clip])
                video.write_videofile(final_video_path, codec='libx264', audio_codec="aac")
            else:
                print('No Images Found '*5)

        clipss = {}
        for z in range(0,len(glob.glob('/content/final_video/*.mp4'))):
            clipss['c{}'.format(z)] = VideoFileClip(glob.glob('/content/final_video/*.mp4')[z])
        clips = []
        for state, capital in clipss.items():
            clips.append(capital)
        final = concatenate_videoclips(clips)
        final_video_path = os.path.join('/content/output/', '{0}_{1}_final-video.mp4'.format(count,cat))
        final.write_videofile(final_video_path, codec='libx264', audio_codec="aac")

already there
46
downloading img no  0
downloading img no  1
downloading img no  2
downloading img no  3
downloading img no  4
downloading img no  5
downloading img no  6
downloading img no  7
downloading img no  8
downloading img no  9
download 10 images
resize error
resize error
resize error
[MoviePy] >>>> Building video /content/video/video0.mp4
[MoviePy] Writing video /content/video/video0.mp4


 83%|████████▎ | 5/6 [00:00<00:00, 41.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/video0.mp4 

[MoviePy] >>>> Building video /content/final_video/final-video0.mp4
[MoviePy] Writing audio in final-video0TEMP_MPY_wvf_snd.mp4


100%|██████████| 1025/1025 [00:02<00:00, 357.98it/s]


[MoviePy] Done.
[MoviePy] Writing video /content/final_video/final-video0.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/final_video/final-video4.mp4 

[MoviePy] >>>> Building video /content/output/CA_e_final-video.mp4
[MoviePy] Writing audio in CA_e_final-videoTEMP_MPY_wvf_snd.mp4


100%|██████████| 3793/3793 [00:10<00:00, 351.63it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/output/CA_e_final-video.mp4



 98%|█████████▊| 43/44 [00:00<00:00, 88.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/output/CA_e_final-video.mp4 



0    https://www.theglobeandmail.com/life/horoscope...
1    https://www.theglobeandmail.com/life/horoscope...
dtype: object